In [3]:
###############################################################
# Purpose :  load baseline adjusted data, PRS, and run genetic models
###############################################################

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, ElasticNet, ElasticNetCV, LogisticRegression, RidgeCV
from sklearn.metrics import r2_score, explained_variance_score, mean_squared_error, f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing

import xgboost as xgb
from xgboost import XGBRegressor
import pickle

import optuna

from step_5_auxiliary_functions import load_data_mgbb, mgb_adj_pheno, preprocess_mgbb_has_genetic_data, load_prsice_mgbb 

In [6]:
# Read in MGB Biobank Phenotype Data
pheno_path = '/2022_BP_ensemble/MGB_phenotypes/'
prs_path = '/2022_BP_ensemble/MGB_PRS_files/MGB_Biobank/'
mgb_biobank_ukbb_prs_df = pd.read_csv(prs_path+'MGB_UKBB+ICBP_SBP_ls-all_R_0.1_500kb.all_score',sep = '\s+')
mgb_biobank_df_3 = pd.read_csv('/2022_BP_ensemble/MGB_phenotypes/BiobankPortal_ts429_2023-05-25-104922.csv', dtype = {'Biobank Subject ID':'str'})

In [7]:
#flag if member on medication or not
mgb_biobank_df_3['On Hypertension Meds'] = np.where((mgb_biobank_df_3['Antihypertensive combinations [Existence (Yes/No)][05/25/2021 to 5/25/2023]']=='Yes') | \
                                       (mgb_biobank_df_3['Antihypertensives-other [Existence (Yes/No)][From 05/25/2021]']=='Yes') | \
                                     (mgb_biobank_df_3['Beta blockers/related [Existence (Yes/No)][From 05/25/2021]']=='Yes') | \
                                     (mgb_biobank_df_3['Calcium channel blockers [Existence (Yes/No)][From 05/25/2021]']=='Yes') |
                                     (mgb_biobank_df_3['Diuretics [Existence (Yes/No)][From 05/25/2021]']=='Yes') | \
                                     (mgb_biobank_df_3['Direct renin inhibitor [Existence (Yes/No)][From 05/25/2021]']=='Yes'),1,0)

In [8]:
mgbb_features_list_1 = ['Biobank Subject ID','Gender','Age','Race',
                        'Patient is consented to biobank [Existence (Yes/No)]',
                       'Body Mass Index (BMI) [Average Value][From 05/25/2021]',
                       'Systolic [Median Value][From 05/25/2021]',
                       'Diastolic [Median Value][From 05/25/2021]']

mgbb_features_list_with_meds = ['Biobank Subject ID','Gender','Age','Race',
                        'Patient is consented to biobank [Existence (Yes/No)]',
                       'Body Mass Index (BMI) [Average Value][From 05/25/2021]',
                       'Systolic [Median Value][From 05/25/2021]',
                       'Diastolic [Median Value][From 05/25/2021]','On Hypertension Meds']
#remove patients on meds
mgb_biobank_df = mgb_biobank_df_3.loc[(mgb_biobank_df_3['Antihypertensive combinations [Existence (Yes/No)][05/25/2021 to 5/25/2023]']=='No') & \
                                       (mgb_biobank_df_3['Antihypertensives-other [Existence (Yes/No)][From 05/25/2021]']=='No') & \
                                     (mgb_biobank_df_3['Beta blockers/related [Existence (Yes/No)][From 05/25/2021]']=='No') & \
                                     (mgb_biobank_df_3['Calcium channel blockers [Existence (Yes/No)][From 05/25/2021]']=='No') &
                                     (mgb_biobank_df_3['Diuretics [Existence (Yes/No)][From 05/25/2021]']=='No') & \
                                     (mgb_biobank_df_3['Direct renin inhibitor [Existence (Yes/No)][From 05/25/2021]']=='No')]

mgbb_combined_df = mgb_biobank_df[mgbb_features_list_1]

In [ ]:
#read in fam file to compare list of patients having their PRS calculated vs those in the phenotype file
mgb_biobank_fam_file = pd.read_csv(pheno_path+"2021—08-11_unrels_pca.eigenvec",\
                                   sep= '\s+')

mgb_biobank_fam_file.head()
mgb_biobank_fam_file.columns = ["FID","Biobank Subject ID","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20"]
#clean up subject ID and only take the second part
mgb_biobank_fam_file['Biobank Subject ID'] = mgb_biobank_fam_file['Biobank Subject ID'].str.split('-').str[1]

#remove duplicate IID by keeping the first value
mgb_biobank_fam_file = mgb_biobank_fam_file.loc[mgb_biobank_fam_file.duplicated(subset=['Biobank Subject ID'])==False]
#remove leading 0s in IID
mgb_biobank_fam_file['Biobank Subject ID'] = [s.lstrip("0") for s in mgb_biobank_fam_file['Biobank Subject ID']]
mgb_biobank_fam_file.head()

print(mgbb_combined_df.shape)
print(mgb_biobank_fam_file.shape)

print(sum(mgb_biobank_fam_file["Biobank Subject ID"].isin(mgbb_combined_df['Biobank Subject ID'])))

In [10]:
#extract the patient id from IID and remove leading 0s
mgb_biobank_ukbb_prs_df_ids = mgb_biobank_ukbb_prs_df['IID'].str.split('-').str[1]
mgb_biobank_ukbb_prs_df_ids = mgb_biobank_ukbb_prs_df_ids.loc[mgb_biobank_ukbb_prs_df_ids.duplicated()==False]
mgb_biobank_ukbb_prs_df_ids = [s.lstrip("0") for s in mgb_biobank_ukbb_prs_df_ids]

In [11]:
mgbb_combined_df.columns = ['sample.id','GENDER','AGE_V1','race_clean',
                            'genomic_data_existence','BMI_V1', 'SBP_V1','DBP_V1']
mgbb_combined_df_clean = mgbb_combined_df

In [12]:
#model parameters based on optuna outputs
sbp_mgb_parameters_baseline = {'max_depth': 4, 'min_child_weight': 40, \
                                           'subsample': 0.7000000000000001, 'colsample_bytree': 0.7000000000000001, \
                                           'lambda': 49, 'alpha': 35, 'gamma': 22, 'eta': 0.04, 'nthread':-1}
sbp_mgb_num_boost_round = 293

dbp_mgb_parameters_baseline = {'max_depth': 100, 'min_child_weight': 9, 'subsample': 0.5,\
                               'colsample_bytree': 0.9, 'lambda': 0, 'alpha': 44, 'gamma': 22, 'eta': 0.01}
dbp_mgb_num_boost_round = 405

In [ ]:
#SBP
sbp_y_mgb, sbp_phenotype_data_mgb = load_data_mgbb(mgbb_combined_df_clean,"SBP_V1")

mgb_data_in_prs_fam = sbp_phenotype_data_mgb.index.isin(mgb_biobank_fam_file["Biobank Subject ID"])
sbp_y_mgb = sbp_y_mgb[mgb_data_in_prs_fam==True]
sbp_phenotype_data_mgb = sbp_phenotype_data_mgb[mgb_data_in_prs_fam==True]
sbp_phenotype_data_mgb['GENDER'] = np.where(sbp_phenotype_data_mgb['GENDER']=="F",1,0)

#train test split and get residuals
x_train_mgbb, x_test_mgbb, y_train_mgbb, y_test_mgbb = train_test_split(sbp_phenotype_data_mgb, \
                                                                        sbp_y_mgb, test_size=0.30, \
                                                                        random_state=1)
#predict baseline phenotype for mgbb
mgb_y_train_resid_sbp, mgb_baseline_results_sbp = mgb_adj_pheno(sbp_phenotype_data_mgb, sbp_y_mgb, model_type = "xgboost", 
                                                                optuna_flag = False, output_results = True,
                                                                params = sbp_mgb_parameters_baseline,
                                                                optuna_num_boost_round = sbp_mgb_num_boost_round,
                                                                save_mod = False, mod_name = 'mgb_baseline_model_weights_sbp'
                                                               )

mgb_y_train_topmed_weights_resid_sbp, mgb_baseline_topmed_results_sbp = mgb_adj_pheno(sbp_phenotype_data_mgb, sbp_y_mgb, model_type = "xgboost", 
                                                                optuna_flag = False, output_results = True,
                                                                params = sbp_mgb_parameters_baseline,
                                                                optuna_num_boost_round = sbp_mgb_num_boost_round, load_baseline_topmed_model = True,
                                                                save_mod = False, mod_name = 'mgb_baseline_model_weights_sbp', 
                                                                                      load_model_name = 'TOPMed_baseline_xgb_model_weights_SBP'
                                                               )

In [ ]:
#DBP
dbp_y_mgb, dbp_phenotype_data_mgb = load_data_mgbb(mgbb_combined_df_clean,"DBP_V1")
dbp_mgb_data_in_prs_fam = dbp_phenotype_data_mgb.index.isin(mgb_biobank_fam_file["Biobank Subject ID"])
dbp_y_mgb = dbp_y_mgb[dbp_mgb_data_in_prs_fam==True]
dbp_phenotype_data_mgb = dbp_phenotype_data_mgb[dbp_mgb_data_in_prs_fam==True]
dbp_phenotype_data_mgb['GENDER'] = np.where(dbp_phenotype_data_mgb['GENDER']=="F",1,0)

#calculate residuals for validation set for TopMed-Trained Model
dbp_mgb_y_train_resid, dbp_baseline_results = mgb_adj_pheno(dbp_phenotype_data_mgb, dbp_y_mgb,model_type = "xgboost", 
                                                            optuna_flag = False, output_results = True,
                                                           params = dbp_mgb_parameters_baseline, 
                                                           optuna_num_boost_round = dbp_mgb_num_boost_round,
                                                           save_mod = True, mod_name = 'mgb_baseline_model_weights_dbp')

mgb_y_train_topmed_weights_resid_dbp, mgb_baseline_topmed_results_dbp = mgb_adj_pheno(dbp_phenotype_data_mgb, dbp_y_mgb, model_type = "xgboost", 
                                                                optuna_flag = False, output_results = True,
                                                                params = dbp_mgb_parameters_baseline,
                                                                optuna_num_boost_round = dbp_mgb_num_boost_round, load_baseline_topmed_model = True,
                                                                save_mod = False, mod_name = 'mgb_baseline_model_weights_dbp', 
                                                                                      load_model_name = 'TOPMed_baseline_dbp_xgb_model_weights_DBP'
                                                               )

#create training and testing set when doing prediction model solely on MGBB
dbp_x_train_mgbb = dbp_phenotype_data_mgb.reindex(x_train_mgbb.index)
dbp_x_test_mgbb = dbp_phenotype_data_mgb.reindex(x_test_mgbb.index)
dbp_y_train_mgbb = dbp_y_mgb.reindex(y_train_mgbb.index)
dbp_y_test_mgbb = dbp_y_mgb.reindex(y_test_mgbb.index)

In [43]:
mgb_biobank_table1_data = pd.concat([sbp_phenotype_data_mgb,sbp_y_mgb,dbp_y_mgb], axis =1)  
mgb_biobank_table1_data.to_csv("/2022_BP_ensemble/MGB_Phenotypes/MGBB_Table1_Data.csv", index=False)


In [46]:
combined_mgb_baseline_results = pd.concat([mgb_baseline_results_sbp,dbp_baseline_results],axis=0)
combined_mgb_baseline_results['phenotype'] = (['SBP']*len(mgb_baseline_results_sbp))+(['DBP']*len(dbp_baseline_results))
combined_mgb_baseline_results.to_csv("/2022_BP_ensemble/Results/MGBB_baseline_results.csv", index=False)

In [48]:
sbp_mgb_biobank_ukbb_prs_df = load_prsice_mgbb("SBP", sbp_phenotype_data_mgb, "UKBB+ICBP", single_threshold = False,
                prs_dir ='/2022_BP_ensemble/MGB_PRS_files/MGB_Biobank/',
                ukbb_flag = False)

sbp_mgb_biobank_mvp_prs_df = load_prsice_mgbb("SBP", sbp_phenotype_data_mgb, "MVP", single_threshold = False,
                prs_dir ='/2022_BP_ensemble/MGB_PRS_files/MGB_Biobank/',
                ukbb_flag = False)

sbp_mgb_biobank_bbj_prs_df = load_prsice_mgbb("SBP", sbp_phenotype_data_mgb, "BBJ", single_threshold = False,
                prs_dir ='/2022_BP_ensemble/MGB_PRS_files/MGB_Biobank/',
                ukbb_flag = False)

In [49]:
#load PRSice for DBP
dbp_mgb_biobank_ukbb_prs_df = load_prsice_mgbb("DBP", dbp_phenotype_data_mgb, "UKBB+ICBP", single_threshold = False,
                prs_dir ='/2022_BP_ensemble/MGB_PRS_files/MGB_Biobank/',
                ukbb_flag = False)
#mgb_biobank_ukbb_prs_df[~mgb_biobank_ukbb_prs_df['SBP_UKBB+ICBP_Pt_0.01'].isnull()]

dbp_mgb_biobank_mvp_prs_df = load_prsice_mgbb("DBP", dbp_phenotype_data_mgb, "MVP", single_threshold = False,
                prs_dir ='/2022_BP_ensemble/MGB_PRS_files/MGB_Biobank/',
                ukbb_flag = False)

dbp_mgb_biobank_bbj_prs_df = load_prsice_mgbb("DBP", dbp_phenotype_data_mgb, "BBJ", single_threshold = False,
                prs_dir ='/2022_BP_ensemble/MGB_PRS_files/MGB_Biobank/',
                ukbb_flag = False)

In [50]:
sbp_phenotype_data_mgb[["race_clean_AA","race_clean_AsA","race_clean_EA","race_clean_HA","race_clean_Other/Unknown"]] = sbp_phenotype_data_mgb\
[["race_clean_AA","race_clean_AsA","race_clean_EA","race_clean_HA","race_clean_Other/Unknown"]].apply(pd.to_numeric)

dbp_phenotype_data_mgb[["race_clean_AA","race_clean_AsA","race_clean_EA","race_clean_HA","race_clean_Other/Unknown"]] = dbp_phenotype_data_mgb\
[["race_clean_AA","race_clean_AsA","race_clean_EA","race_clean_HA","race_clean_Other/Unknown"]].apply(pd.to_numeric)

In [51]:
#find which rows has missing UKBB PRS data in mgb_biobank_ukbb_prs_df (loaded in from load_prsice_mgbb)

topmed_model_3_cols = ['race_clean_AA', 'race_clean_AsA', 'race_clean_EA', 'race_clean_HA',
       'race_clean_Other/Unknown', 'GENDER', 'AGE_V1', 'BMI_V1',
       'SBP_BBJ_Pt_5e-08', 'SBP_BBJ_Pt_1e-07', 'SBP_BBJ_Pt_1e-06',
       'SBP_BBJ_Pt_1e-05', 'SBP_BBJ_Pt_0.0001', 'SBP_BBJ_Pt_0.001',
       'SBP_BBJ_Pt_0.01', 'SBP_MVP_Pt_5e-08', 'SBP_MVP_Pt_1e-07',
       'SBP_MVP_Pt_1e-06', 'SBP_MVP_Pt_1e-05', 'SBP_MVP_Pt_0.0001',
       'SBP_MVP_Pt_0.001', 'SBP_MVP_Pt_0.01', 'SBP_UKBB+ICBP_Pt_5e-08',
       'SBP_UKBB+ICBP_Pt_1e-07', 'SBP_UKBB+ICBP_Pt_1e-06',
       'SBP_UKBB+ICBP_Pt_1e-05', 'SBP_UKBB+ICBP_Pt_0.0001',
       'SBP_UKBB+ICBP_Pt_0.001', 'SBP_UKBB+ICBP_Pt_0.01']

#SBP
ukbb_sna = sbp_mgb_biobank_ukbb_prs_df.isna().any(axis = 1)
# model_1_x_data = phenotype_data_mgb
sbp_model_1_x_data = pd.concat([sbp_phenotype_data_mgb, sbp_mgb_biobank_ukbb_prs_df["SBP_UKBB+ICBP_Pt_0.01"]], axis=1)[ukbb_sna==False]
sbp_model_2_x_data = pd.concat([sbp_phenotype_data_mgb, sbp_mgb_biobank_ukbb_prs_df], axis=1)[ukbb_sna==False]
sbp_model_3_x_data = pd.concat([sbp_phenotype_data_mgb, sbp_mgb_biobank_ukbb_prs_df, \
                            sbp_mgb_biobank_bbj_prs_df, \
                            sbp_mgb_biobank_mvp_prs_df], axis=1)[ukbb_sna==False] #mgb_biobank_mvp_prs_df
sbp_model_3_x_data = sbp_model_3_x_data[topmed_model_3_cols]

#DBP
#find which rows has missing UKBB PRS data in mgb_biobank_ukbb_prs_df (loaded in from load_prsice_mgbb)
dbp_ukbb_sna = dbp_mgb_biobank_ukbb_prs_df.isna().any(axis = 1)
# dbp_model_1_x_data = dbp_phenotype_data_mgb
dbp_model_1_x_data = pd.concat([dbp_phenotype_data_mgb, dbp_mgb_biobank_ukbb_prs_df["DBP_UKBB+ICBP_Pt_0.01"]], axis=1)[dbp_ukbb_sna==False]
dbp_model_2_x_data = pd.concat([dbp_phenotype_data_mgb, dbp_mgb_biobank_ukbb_prs_df], axis=1)[dbp_ukbb_sna==False]
dbp_model_3_x_data = pd.concat([dbp_phenotype_data_mgb, dbp_mgb_biobank_ukbb_prs_df, \
                            dbp_mgb_biobank_bbj_prs_df, \
                            dbp_mgb_biobank_mvp_prs_df], axis=1)[dbp_ukbb_sna==False] #mgb_biobank_mvp_prs_df

In [54]:
#output dataset for covariates, outcome phenotype, residuals of outome predictions trained on MGBB, 
#residuals of outcome predictions trained on TOPMed weights
sbp_phenotype_data_mgb.to_csv(pheno_path+'SBP_MGBB_Train.csv') #covariates from validation dataset (MGBB)
sbp_y_mgb.to_csv(pheno_path+'SBP_MGBB_Y_Train.csv') #validation target variable dataset
#trained baseline model on MGB and validate on the same dataset
mgb_y_train_resid_sbp.to_csv(pheno_path+'SBP_MGBB_Y_Train_Resid.csv') #baseline predictions residuals when trained on MGBB

#trained baseline model on TOPMed and validated on MGBB
mgb_y_train_topmed_weights_resid_sbp.to_csv(pheno_path+'SBP_MGBB_TOPMED_WEIGHTS_Y_Train_Resid.csv') #prediction residuals from TOPMed baseline model weights

#output datasets for PRSs
sbp_mgb_biobank_ukbb_prs_df.to_csv(prs_path+'mgbb_sbp_ukbb_prs.csv')
sbp_mgb_biobank_bbj_prs_df.to_csv(prs_path+'mgbb_sbp_bbj_prs.csv')
sbp_mgb_biobank_mvp_prs_df.to_csv(prs_path+'mgbb_sbp_mvp_prs.csv')

#repeat the same as above but for DBP
dbp_phenotype_data_mgb.to_csv(pheno_path+'DBP_MGBB_Train.csv')
dbp_y_mgb.to_csv(pheno_path+'DBP_MGBB_Y_Train.csv')
dbp_mgb_y_train_resid.to_csv(pheno_path+'DBP_MGBB_Y_Train_Resid.csv')
mgb_y_train_topmed_weights_resid_dbp.to_csv(pheno_path+'DBP_MGBB_TOPMED_WEIGHTS_Y_Train_Resid.csv')

dbp_mgb_biobank_ukbb_prs_df.to_csv(prs_path+'mgbb_dbp_ukbb_prs.csv')
dbp_mgb_biobank_bbj_prs_df.to_csv(prs_path+'mgbb_dbp_bbj_prs.csv')
dbp_mgb_biobank_mvp_prs_df.to_csv(prs_path+'mgbb_dbp_mvp_prs.csv')